# SolarX Sizing Example

This notebook sizes a SOLARX plant and runs the Energy Management System (EMS) optimization for a specific site. We will:

1. install/load `hydesign`
2. set site + data files
3. configure the model and decision variables
4.  run the Efficient Global Optimization (EGO) search to maximize an objective (in this example: `NPV_over_CAPEX`).

## Installation and imports

In this section, we:
* Checks if `hydesign` is installed; if not, installs the repo from GitLab.
* Imports the SOLARX assembly (`hpp_model_solarX`) and the EGO driver.

Install hydesign if not already

In [1]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git   

Import required modules

In [ ]:
import os
from hydesign.assembly.hpp_assembly_solarX import hpp_model_solarX as hpp_model
from hydesign.examples import examples_filepath
from hydesign.Parallel_EGO import EfficientGlobalOptimizationDriver

## Define the site

In this section, we will:

* Sets a site name and coordinates (Spain example).
* Points to the time-series CSV (`input_ts_fn`) and a YAML with simulation parameters (`sim_pars_fn`).

In [3]:
name = 'Spain_good_solar'

longitude = -5.99
latitude = 37.39
altitude = 10

In [ ]:
#time series
input_ts_fn = os.path.join(examples_filepath, 'Europe/solarX_Spain.csv')

# YAML simulation parameters
sim_pars_fn = os.path.join(examples_filepath, 'Europe/hpp_pars_solarX.yml')

## Configure the optimization run

In this section, we will build the `inputs` dictionary that configures the optimization run:

  * `opt_var`: objective (here normalized NPV).
  * `n_procs`: parallel processes for evaluations.
  * `n_doe`: initial design-of-experiments samples.
  * `n_clusters`: surrogate/model clustering (speed vs fidelity).
  * `max_iter`: EGO iterations after DOE.
  * `npred`: prediction budget for infill (trade-off speed vs quality).
  * Tolerances and convergence controls.
  * `hpp_model`: the SOLARX assembly we imported from solarX module.
  * `batch_size`: passed into the EMS to limit solve size.

>Note: We set `batch_size = 1 * 24` (1 days hourly = **24 steps**) to run with the community version of Cplex. If you run with linsences version of CPLEX increase the batch size for a more realistic results (e.g., `batch_size = 3*30*24` or `6*30*24`).

In [5]:
batch_size = 1 * 24

inputs = {
    'example': None,
    'name': name,
    'longitude': longitude,
    'latitude': latitude,
    'altitude': altitude,
    'input_ts_fn': input_ts_fn,
    'sim_pars_fn': sim_pars_fn,

    'opt_var': "NPV_over_CAPEX",
    'num_batteries': 1,
    'n_procs': 5,
    'n_doe': 40,
    'n_clusters': 4,
    'n_seed': 0,
    'max_iter': 15,
    'final_design_fn': 'hydesign_design_0.csv',
    'npred': 3e4,
    'tol': 1e-6,
    'min_conv_iter': 2,
    'work_dir': './',
    'hpp_model': hpp_model,
    'batch_size': batch_size,
    }

## Set sizing ranges/values

In this section, we will define the design variables (and fixed values) for the optimizer under `inputs['variables']`.
* Each entry can be:

  * `{'var_type': 'design', 'limits': [min, max], 'types': 'float|int'}` → optimizer will choose.
  * `{'var_type': 'fixed', 'value': ...}` → held constant.

**In this specific example:**

* Solar field area is fixed at 8000.
* Tower height fixed at 50 (comment notes allowed discrete values).
* `tower_num` is a **design variable** in `[1, 30]` (integer).
* CPV/CST/hydrogen receiver areas and thermal/power capacities are **design variables** with ranges, except biogas/H₂ reactors are fixed to 0 (disabled) in this example.

>**Tips:**
>
>* Keep ranges realistic; wide ranges slow convergence.
>* Use `int` for discrete counts (`tower_num`).
>* If you don’t model a subsystem, set its area/capacity to a fixed `0` to reduce problem size and speed up runs.

In [ ]:
inputs['variables'] = {
    # sf
    'sf_area':
        # {'var_type':'design',
        #   'limits':[1e3, 1e4],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
            'value': 8000
            },
    'tower_height':
        # {'var_type':'design',
        #   'limits':[11, 30],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
            'value': 50 # can only be 20, 25 or 30
            },
    'tower_num':
        {'var_type':'design',
            'limits':[1, 30],
            'types':'int'
            },
        # {'var_type':'fixed',
        #  'value': 25 # can only be 20, 25 or 30
        #  },

    # cpv
    'area_cpv_receiver_m2':
        {'var_type':'design',
            'limits':[0, 20],
            'types':'float'
            },
        # {'var_type':'fixed',
        #  'value': 4
        # },

    # cst
    'heat_exchanger_capacity':
        {'var_type':'design',
            'limits':[0, 200],
            'types':'float'
            },
        # {'var_type':'fixed',
        #  'value': 400
        # },
    'p_rated_st':
        {'var_type':'design',
            'limits':[0, 50],
            'types':'float'
            },
        # {'var_type':'fixed',
        #  'value': 12.5
        # },
    'v_molten_salt_tank_m3':
        {'var_type':'design',
            'limits':[0, 1e4],
            'types':'float'
            },
        # {'var_type':'fixed',
        #  'value': 2300
        # },
    'area_cst_receiver_m2':
        {'var_type':'design',
            'limits':[0, 20],
            'types':'float'
            },
        # {'var_type':'fixed',
        #  'value': 9
        # },

    # bigas_h2
    'area_dni_reactor_biogas_h2':
        # {'var_type':'design',
        #   'limits':[0, 20],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
            'value': 0
        },
    'area_el_reactor_biogas_h2':
        # {'var_type':'design',
        #   'limits':[0, 200],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
            'value': 0
        },

            }


## Run sizing + ems optimization

In this section, we will create the EGO driver and runs the sizing loop optimization for the hybrid power plant.
* The driver evaluates the model, trains surrogates, selects new points, and iterates up to `max_iter`.

>**Performance notes:**
>
>* `n_procs=5` runs evaluations in parallel (good for multi-core CPUs).
>* Large `n_doe` and `max_iter` increase runtime; tune for your machine.
>* If you see **CPLEX CE 1016 errors**, lower `batch_size` or use a licensed CPLEX.

In [7]:
EGOD = EfficientGlobalOptimizationDriver(**inputs)
EGOD.run()





Sizing a HPP plant at Spain_good_solar:


Fixed parameters on the site
-------------------------------
longitude = -5.99
latitude = 37.39
altitude = 10

input_ts_fn length is not a complete number of years (hyDesign handles years as 365 days).
The file has been modified and stored in ./input_ts_modified.csv



Initial 40 simulations took 7.16 minutes
  Current solution -NPV_over_CAPEX = -4.431E-01
  Current No. model evals: 40

Update sm and extract candidate points took 0.15 minutes
Check-optimal candidates: new 16 simulations took 2.91 minutes
  Current solution -NPV_over_CAPEX = -8.060E-01
  Current No. model evals: 56
  rel_yopt_change = -4.50E-01
Iteration 1 took 3.09 minutes

Update sm and extract candidate points took 0.14 minutes
Check-optimal candidates: new 16 simulations took 2.64 minutes
  Current solution -NPV_over_CAPEX = -1.078E+00
  Current No. model evals: 70
  rel_yopt_change = -2.53E-01
Iteration 2 took 2.83 minutes

Update sm and extract candidate points took 0.

Get the result

In [8]:
EGOD.result

,sf_area,tower_height,tower_num,area_cpv_receiver_m2,heat_exchanger_capacity,p_rated_st,v_molten_salt_tank_m3,area_cst_receiver_m2,area_dni_reactor_biogas_h2,area_el_reactor_biogas_h2,...,penalty lifetime [MEuro],AEP [GWh],AH2P [T],GUF,Total curtailment [GWh],Break-even PPA price el [Euro/MWh],Break-even PPA price H2 [Euro/kg],design obj,opt time [min],n_model_evals
Spain_good_solar,8000.0,50.0,30.0,8.8591,144.601701,2.7925,49.5,0.745,0.0,0.0,...,0.0,161.317623,0.0,1.841525,0.0,34.811364,50.0,NPV_over_CAPEX,36.42,174
